In [1]:
from bioprospecting import structural_search
from bioprospecting import alignment
from bioprospecting import PDB 
import os
import shutil
import glob
import pandas as pd
from Bio import PDB as pdb


In [17]:
folder = os.getcwd()

cluster_srch = folder.split("/")[-1]
path = f"/home/ivan/BSC/Immob/Proyectos/CBMs/CAZY/all_cbm_clusters/{cluster_srch}_*"

matching_folders = glob.glob(path)

if matching_folders:
    target = matching_folders[0]
    sorted_files = sorted(os.listdir(target))
    my_file = sorted_files[0]

    print(f"Accediendo a directorio {target}.")
    pdb_path = os.path.join(target, my_file)
    
    shutil.copy(pdb_path, folder)
    print(f"El archivo {my_file} ha sido movido al directorio de trabajo.")
    new_path = os.path.join(folder, my_file)   

In [18]:
FS_search = structural_search.foldseek.runEasySearch(target_pdb=new_path, output_folder="FoldSeek_Search", db="/home/bubbles/foldseek_databases/pdb")

NameError: name 'new_path' is not defined

In [3]:
path = "/home/idg29/BSC/Immob/Proyectos/Try/0042/FoldSeek_Search/fs_output.txt"

pdb_id = pd.read_csv(path, sep = "\t")

pdb_id.columns=["OG_Structure", "PDB_id", "C1", "C2", "C3", "C4", "C5", "OG_Seq", "Sim_Seq"]

In [4]:
#Dicionario con PDB_ID, Cadena de interés y secuencia.
pdb_dic = {}
pdb_list = []
for ident in pdb_id["PDB_id"]:
    chain = (ident[-1])
    pdb_dic[ident]={}
    for seq in pdb_id.loc[pdb_id["PDB_id"]==ident]["Sim_Seq"]:
        pdb_dic[ident][chain]=seq

#Lista tan solo de los PDB_ID
for PDB_ident in pdb_dic:
    pdb_list.append((PDB_ident[:4]).upper())
    

In [5]:
pdb_dic

{'3d85_A': {'A': 'DIVMTQSPATLSVTPGDRVSLSCRASQSISDYLHWYRQKSHESPRLLIKYASQSISGIPSRFSGSGSGSDFTLSINSVEPEDVGVYYCQNGHSFPFTFGSGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRGE'},
 '8a1e_E': {'E': 'EIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRLLIYDASNRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYSCQQRNNWPPTFGGGTKVEIKRA'},
 '4n1e_B': {'B': 'DIQMTQSPSSLSASVGDRVTITCRASQDIAAALNWYQQKPGKAPKLLIYASSYLQSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYCQQDGYYPATFGQGTKVEIK'},
 '3giz_L': {'L': 'EIVLTQSPATLSLSPGERATLSCRASQSVSSYLAWYQQKPGQAPRLLIYDASNRATGIPARFSGSGSGTDFTLTISSLEPEDFAVYYCQQRSNWPITFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR'},
 '6p4c_L': {'L': 'DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATPIVKSFNRNE'},
 '2yc1_E': {'E': 'SEIVLTQSPATLSVSPGERATLSCR

In [ ]:
#Descargar PDBs
Dwnld_PDBS = PDB.retrievePDBs(pdb_list)

In [23]:
PDB_fold = "/home/idg29/BSC/Immob/Proyectos/Try/0042/PDB"

if not os.pprint(chain.id)
                        print(key)ath.exists(PDB_fold+"/PDB_Chain_Select"):
    os.mkdir(PDB_fold+"/PDB_Chain_Select")

parser = pdb.PDBParser(QUIET=True)
for PDBB in os.listdir(PDB_fold):
    if PDBB.endswith(".pdb"):
        for key in pdb_dic:
            if (PDBB.split(".")[0]).lower() in key:                
                structure = parser.get_structure(PDBB, os.path.join(PDB_fold, PDBB))
                chain_id = (key.split(".")[0]).split("_")[-1]
                for model in structure:
                    for chain in model:
                        if chain.id == chain_id:
                            new_structure = PDB.chainsAsStructure(chain)
                            PDB.saveStructureToPDB(new_structure, output=f"{PDB_fold}/PDB_Chain_Select/{PDBB}_{chain_id}")

        
    

In [ ]:
PDB_fold = "/home/ivan/BSC/Immob/Proyectos/Try/0042/PDB"
Clustering = alignment.foldseek.structuralClustering(job_folder="Clustering_Fold", models_folder=PDB_fold)
